# Image Classification with Kimia Database

**Author:** Oussama GUELFAA  
**Date:** 01-04-2025

This notebook demonstrates image classification using the Kimia database. We'll extract features from binary images and use machine learning techniques to classify them.

## 38.1. Feature Extraction

The image database used in this tutorial is composed of 18 classes. Each class contains 12 images. All these 216 images come from the Kimia database. In order to classify these images, we first have to extract some features of each binary image.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from skimage import measure, io
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns

In [ ]:
# Define the dataset path and classes
import glob
# Use the correct path to the Kimia dataset
rep = '../../data/images_Kimia/'
classes = ['bone', 'apple', 'camel']  # We only have these 3 classes in our dataset
    
# If you have the full Kimia216 dataset, uncomment these classes:
# classes = ['bird', 'bone', 'brick', 'camel', 'car', 'children',
#           'classic', 'elephant', 'face', 'fork', 'fountain',
#           'glass', 'hammer', 'heart', 'key', 'misk', 'ray', 'turtle']

nbClasses = len(classes)
nbImages = 20  # Each class has 20 images in our dataset

# The features are manually computed
properties = np.zeros((nbClasses*nbImages, 9))
target = np.zeros(nbClasses * nbImages)
index = 0

for ind_c, c in enumerate(classes):
    filelist = glob.glob(rep+c+'*')
    for filename in filelist:
        print(filename)

### Feature Extraction Function

For each image in the database, we'll extract a number of different features using scikit-image's `regionprops` function.

In [ ]:
def extract_features(image_path):
    """Extract region properties from a binary image."""
    # Read the image
    img = io.imread(image_path, as_gray=True)
    
    # Ensure binary image
    if img.max() > 1:
        img = img > 128
    
    # Extract region properties
    props = measure.regionprops(measure.label(img))[0]
    
    # Extract features
    features = np.array([
        props.area,                      # Area of the region
        props.perimeter,                 # Perimeter of the region
        props.eccentricity,              # Eccentricity of the region
        props.equivalent_diameter_area,  # Diameter of circle with same area
        props.euler_number,              # Euler number
        props.extent,                    # Ratio of pixels in region to pixels in bounding box
        props.major_axis_length,         # Length of major axis
        props.minor_axis_length,         # Length of minor axis
        props.solidity                   # Ratio of pixels in the region to pixels in the convex hull
    ])
    
    return features

In [ ]:
# Extract features for all images
properties = np.zeros((nbClasses*nbImages, 9))
target = np.zeros(nbClasses * nbImages)
index = 0

for ind_c, c in enumerate(classes):
    filelist = glob.glob(rep+c+'*')
    for filename in filelist:
        try:
            # Extract features
            features = extract_features(filename)
            properties[index] = features
            target[index] = ind_c
            index += 1
        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Check the shape of our feature array
print(f"Feature array shape: {properties.shape}")
print(f"Target array shape: {target.shape}")

## 38.2. Image Classification

In order to classify the images, we are going to use neural networks. Pattern recognition networks are feedforward networks that can be trained to classify inputs according to target classes. The inputs are the features of each image. The target data are the labels, indicating the class of each image.

### 38.2.1. Construction of the Array of Properties

- Build the target data, representing the class of each image. The target data for pattern recognition networks should consist of one vector containing the index of the target class. In our example, the target data will be a vector of 216 elements.
- The database will be divided into a training set (75%) and a test set (25%).

In [ ]:
# Split the data into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    properties, target, test_size=0.25, random_state=42, stratify=target)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

### 38.2.2. Training and Classification

- Run the training task and classify the test images.
- Show the classification confusion matrix as well as the overall performance.

In [ ]:
# Normalize the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Train a neural network classifier
from sklearn.neural_network import MLPClassifier

# Create and train the classifier
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', 
                    solver='adam', max_iter=1000, random_state=42)
mlp.fit(X_train_scaled, y_train)

# Make predictions
y_pred = mlp.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print classification report
print("\nClassification Report:")
# Convert numeric labels to string labels for the report
class_labels = [str(i) for i in range(len(classes))]
print(classification_report(y_test, y_pred, target_names=class_labels))

In [ ]:
# Plot confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(12, 10))
# Use class_labels for the confusion matrix visualization
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix (Accuracy: {accuracy:.2f})')
plt.tight_layout()
plt.show()

### Try with SVM Classifier

Let's also try using an SVM classifier to compare performance.

In [ ]:
# Train an SVM classifier
from sklearn.svm import SVC

# Create and train the classifier
svm = SVC(C=1.0, kernel='rbf', gamma='scale', random_state=42)
svm.fit(X_train_scaled, y_train)

# Make predictions
y_pred_svm = svm.predict(X_test_scaled)

# Calculate accuracy
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"SVM Accuracy: {accuracy_svm:.2f}")

# Print classification report
print("\nSVM Classification Report:")
# Use the same class labels as before
print(classification_report(y_test, y_pred_svm, target_names=class_labels))

In [ ]:
# Plot SVM confusion matrix
cm_svm = confusion_matrix(y_test, y_pred_svm)

plt.figure(figsize=(12, 10))
# Use class_labels for the confusion matrix visualization
sns.heatmap(cm_svm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'SVM Confusion Matrix (Accuracy: {accuracy_svm:.2f})')
plt.tight_layout()
plt.show()

### Improving Classification Performance

Let's try to change the parameters of the neural network to improve the classification performance.

In [ ]:
# Try a more complex neural network
mlp_improved = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', 
                            solver='adam', max_iter=2000, alpha=0.0001,
                            learning_rate='adaptive', random_state=42)
mlp_improved.fit(X_train_scaled, y_train)

# Make predictions
y_pred_improved = mlp_improved.predict(X_test_scaled)

# Calculate accuracy
accuracy_improved = accuracy_score(y_test, y_pred_improved)
print(f"Improved MLP Accuracy: {accuracy_improved:.2f}")

# Print classification report
print("\nImproved MLP Classification Report:")
# Use the same class labels as before
print(classification_report(y_test, y_pred_improved, target_names=class_labels))

In [ ]:
# Plot improved MLP confusion matrix
cm_improved = confusion_matrix(y_test, y_pred_improved)

plt.figure(figsize=(12, 10))
# Use class_labels for the confusion matrix visualization
sns.heatmap(cm_improved, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Improved MLP Confusion Matrix (Accuracy: {accuracy_improved:.2f})')
plt.tight_layout()
plt.show()

## Conclusion

In this notebook, we've demonstrated how to:

1. Extract meaningful features from binary images using region properties
2. Organize these features into a dataset suitable for machine learning
3. Train different classifiers (MLP and SVM) on the dataset
4. Evaluate the performance of the classifiers
5. Improve the classification performance by tuning parameters

The results show that both neural networks and SVMs can effectively classify the Kimia database images based on the extracted features.